In [1]:
%matplotlib inline
from skimage import io
from skimage.future import graph
from skimage.morphology import watershed
from skimage.io import imread
from matplotlib import pyplot as plt
import numpy as np
import time
import datetime
import os, sys
from os.path import isdir,join,isfile
from scipy.optimize import linear_sum_assignment
from scipy.optimize import curve_fit



In [ ]:
# function for tracking same masks by maximum overlap
pathToDataBase = "C:/Users/dipsi/Desktop/PhD_docs/Instance_segmentation/collection/salad_10_days"
pathToDataBaselabeled = "C:/Users/dipsi/Desktop/PhD_docs/Instance_segmentation/collection/salad_10_days_newlabel"
saladlist = os.listdir(pathToDataBase)
masklist = os.listdir(pathToDataBaselabeled)
for i in range(45,len(saladlist)-1):
    path = os.path.join(pathToDataBase, saladlist[i])
    path = path + "/photo1s_label.png"
    path1 = os.path.join(pathToDataBaselabeled, masklist[-1])
    img1 = io.imread(path1)
    img2 = io.imread(path)
    ls1 = np.unique(img1)[1:]
    ls2 = np.unique(img2)[1:]
    l2id_1 = {l:i for i,l in enumerate(ls1)}
    l2id_2 = {l:i for i,l in enumerate(ls2)}
    inter = np.zeros((len(ls1), len(ls2)))
    m1 = np.ma.masked_array(img1, img1==0)
    m2 = np.ma.masked_array(img2, img2==0)
    for l1 in ls1:
        for l2 in ls2:
            inter[l2id_1[l1],l2id_2[l2]] = np.sum(np.logical_and(m1 == l1, m2 == l2))

    assigned_i1, assigned_i2 = linear_sum_assignment(-inter)
    ls2_left = set(ls2).difference(ls2[assigned_i2])
    ls2_left_new = ls1[assigned_i1][-1] + np.arange(1, len(ls2_left) + 1)
    newimg2 = img2.copy()
    for l1, l2 in zip(ls1[assigned_i1], ls2[assigned_i2]):

        newimg2[img2 == l2] = l1
    
    for l2_left, l2_left_new in zip(ls2_left,ls2_left_new):

        newimg2[img2 == l2_left] = l2_left_new
    io.imsave("C:/Users/dipsi/Desktop/PhD_docs/Instance_segmentation/collection/salad_10_days_newlabel/{}.png".format(i-43), newimg2) 
    

In [ ]:
# function for tracking same masks by maximum overlap with treatment


basepath = "C:/Users/dipsi/Desktop/PhD_docs/Instance_segmentation/salad_10_days_filt/"

pathToDataBase = basepath+"salad_10_days_6thplant_filtered"

saladlist = sorted([f for f in os.listdir(pathToDataBase) if isdir(join(pathToDataBase,f))])


n = 6

labels = []
n_objects = []
for p in saladlist:
    img = imread(join(pathToDataBase,p,"photo{0}s_label.png".format(n)))
    labels.append(img)
    n_objects.append(np.max(img))
labels = np.dstack(labels)
     
def parse_next(ref,new, max_resize=0.7):
    n_ref = np.max(ref)
    n_new = np.max(new)
    
    new_ref = np.zeros_like(new)
    
    cost_mrx = np.zeros((n_ref,n_new))
    
    
    for i in range(n_ref):
        for j in range(n_new):
            cost_mrx[i,j] = np.sum(np.logical_and(ref == i+1, new == j+1))/np.maximum(float(np.sum(ref==i+1)),1.)
    
    #print(cost_mrx)
    #Все хорошо нам надо на new помять индексы по анологии с ref
    if n_new==n_ref:
        #print("same",cost_mrx)
        ii, jj = linear_sum_assignment(-cost_mrx)
        #print(ii,jj)
        for i,j in zip(ii,jj):
            new_ref[new==j+1]=i+1
        
        
    if n_new>n_ref:
        #print("more!")
        #Надо проверить не ошибка ли, если нет - все ок
        ii, jj = linear_sum_assignment(-cost_mrx)
        uq = np.unique(new).tolist()
        uq.remove(0)
        for i,j in zip(ii,jj):
            if cost_mrx[i,j]<max_resize:
                #видимо ошибка
                for k in range(0,cost_mrx.shape[1]):
                    if i==np.argmax(cost_mrx[:,k]):
                        new_ref[new==k+1]=i+1
                        uq.remove(k+1)
            else:
                new_ref[new==j+1]=i+1
                uq.remove(j+1)
        for u in uq:
            
            if 0<np.sum(new==u):
                print(u)
                new_ref[new==u]=np.max(new_ref)+1
            
    
    #Уменьшилось кол-во объектов
    if n_new<n_ref:
        #Что-то не так надо искать кто склеелся - будем резать ближайшими соседями
        new_ref[new>0]=1
        new_ref = watershed(new_ref, ref, mask=new>0)
        
    return new_ref,cost_mrx
    
    
start = 0
corrected = [labels[:,:,start]]
matrix = []
for i in range(start+1,labels.shape[2]):
    r,m = parse_next(corrected[-1],labels[:,:,i])
    corrected.append(r)
    matrix.append(m)
    
    
dc = np.dstack(corrected)
reference = 545
for i in range(1,5):
    z = np.sum(dc==i,axis=(0,1))/reference
    plt.plot(z)

In [ ]:
# leaf area calculation
Area = list()
for i in range(2,len(os.listdir(pathToDataBaselabeled))):
    imag = io.imread("C:/Users/dipsi/Desktop/PhD_docs/Instance_segmentation/collection/salad_10_days_newlabel/{}.png".format(i))
    Areacurr = list()
    for j in range(1, len(np.unique(imag))):
        Areacurr.append(np.sum(imag == j))
    ar = np.asarray(Areacurr)
    Area.append(ar)

In [ ]:
# plotting area of tracked masks in pixels and model fitting without treatment

basepath = "C:/Users/dipsi/Desktop/PhD_docs/Instance_segmentation/salad_10_days_filt/"

pathToDataBase = basepath+"salad_10_days_6thplant_filtered"

n=6


saladlist = os.listdir(pathToDataBase)
timeArr = list()
for item in saladlist:
    path = os.path.join(pathToDataBase, item)
    pathToPhoto = os.path.join(path, "photo%(number)s" % {"number":n})
    pathToPhoto = pathToPhoto + "s"
        
    t = (2018, 9, int(float(item[13:15])), int(float(item[16:18])), int(float(item[19:21])), 38, 1, 1, 0)       
    timeArr.append(time.mktime(t))

for i in range(1,len(timeArr)):
    timeArr[i]=(timeArr[i] - timeArr[0])/86400

timeArr[0]=0
reference = 545
for i in range(1,5):
    z = np.sum(dc==i,axis=(0,1))/reference
    plt.plot(timeArr,z)


growth1 = np.sum(dc==3,axis=(0,1))/reference
growth2 = np.sum(dc==4,axis=(0,1))/reference

ii1=np.nonzero(growth1)
ii2=np.nonzero(growth2)

time1=np.asarray(timeArr)
time2=np.asarray(timeArr)
timeed1=np.zeros(len(ii1[0]))
timeed2=np.zeros(len(ii2[0]))
timeed1[0]=0
timeed1[1:]= time1[ii1][1:] - time1[ii1][0]
timeed2[0]=0
timeed2[1:]= time2[ii2][1:] - time2[ii2][0]

print(growth1[ii1])
print(growth2[ii2])


def func1(t, m):
    Smax = 100
    S0=0.1
    return (Smax*S0*np.exp(m*t))/(Smax+S0*(np.exp(m*t)-1.0))
def func2(t, m):
    Smax = 100
    S0=0.1
    return (Smax*S0*np.exp(m*t))/(Smax+S0*(np.exp(m*t)-1.0))

popt1, pconv1 = curve_fit(func1, timeed1, growth1[ii1])
popt2, pconv2 = curve_fit(func2, timeed2, growth2[ii2])

print(popt1)
print(popt2)